In [2]:
# importing libraries for data preprocessing
import pandas as pd 
import numpy as np
from sklearn.preprocessing import MinMaxScaler

#importing libraries for neural network model
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM

# import libraries for plotting
import cufflinks as cf
import plotly.offline
from  plotly.offline import plot_mpl
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# import libraries for error metrics
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


/home/cooldudeanmol/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# data preprocessing
# reading dataset
data = pd.read_csv("PowerData.csv",index_col=0)
data = data.loc[:'2010-01-01']
data.head()

,IPG2211A2N
DATE,
1939-01-01,3.3842
1939-02-01,3.4100
1939-03-01,3.4875
1939-04-01,3.5133
1939-05-01,3.5133


In [4]:
# making index datetime
data.index = pd.to_datetime(data.index)


In [5]:
# chainging column name
data.columns = ['Energy Production']
data.head()

,Energy Production
DATE,
1939-01-01,3.3842
1939-02-01,3.4100
1939-03-01,3.4875
1939-04-01,3.5133
1939-05-01,3.5133


In [6]:
print('Min', np.min(data))
print('Max', np.max(data))

Min Energy Production    3.3842
dtype: float64
Max Energy Production    119.2473
dtype: float64


In [7]:
# normalizing the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(data)

In [8]:
#plotting the data 
data.iplot(title="power", xTitle='TIme Stamp',yTitle='Demand')

In [9]:
print('Min', np.min(scaled))
print('Max', np.max(scaled))

Min 0.0
Max 1.0


In [10]:
len(scaled)
# !pip3 install pyramid-arima


853

In [11]:
#create train/test datasets

train_size = int(len(scaled)*.7)
test_size = int(len(scaled - train_size))
train, test = scaled[:train_size, :], scaled[train_size:, :]

print(len(train), len(test))

597 256


In [12]:
#changing data into required window format  
def create_dataset(dataset, look_back = 1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[(i+look_back), 0])
        
    return np.array(dataX), np.array(dataY)

In [13]:
look_back = 12
Xtrain, Ytrain = create_dataset(train, look_back)
Xtest, Ytest = create_dataset(test, look_back)

In [14]:
Xtrain = np.reshape(Xtrain, (Xtrain.shape[0], Xtrain.shape[1], 1))
Xtest = np.reshape(Xtest, (Xtest.shape[0], Xtest.shape[1], 1))
print(Xtrain.shape)
print(Xtest.shape)

(584, 12, 1)
(243, 12, 1)


In [15]:
# Neural network model
batch_size = 1

# model = Sequential()
# layers = [1, 75, 100, prediction_steps]
# model.add(LSTM(layers[1], input_shape=(None, layers[0]), return_sequences=True))  # add first layer
# model.add(Dropout(0.2))  # add dropout for first layer
# model.add(LSTM(layers[2], return_sequences=False))  # add second layer
# model.add(Dropout(0.2))  # add dropout for second layer
# model.add(Dense(layers[3]))  # add output layer
# model.add(Activation('linear'))  # output layer with linear activation
    
    
# model = Sequential()
# model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=1))
# model.add(Dense(1))
# model.compile(loss = 'mean_squared_error', optimizer='adam')
# model.fit(Xtrain, Ytrain, epochs=100, batch_size=batch_size, verbose = 2, shuffle=True)

model = Sequential()
model.add(LSTM(75, input_shape=(None, 1), return_sequences=True))  # add first layer
model.add(Dropout(0.2))  # add dropout for first layer
model.add(LSTM(100, return_sequences=False))  # add second layer
model.add(Dropout(0.2))  # add dropout for second layer
model.add(Dense(1))
model.compile(loss = 'mean_squared_error', optimizer='adam')
model.fit(Xtrain, Ytrain, epochs=50, batch_size=batch_size, verbose = 2, shuffle=True)





Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
 - 11s - loss: 0.0036
Epoch 2/50
 - 7s - loss: 0.0022
Epoch 3/50
 - 7s - loss: 0.0018
Epoch 4/50
 - 7s - loss: 0.0018
Epoch 5/50
 - 7s - loss: 0.0020
Epoch 6/50
 - 7s - loss: 0.0019
Epoch 7/50
 - 7s - loss: 0.0015
Epoch 8/50
 - 7s - loss: 0.0016
Epoch 9/50
 - 9s - loss: 0.0014
Epoch 10/50
 - 7s - loss: 0.0015
Epoch 11/50
 - 8s - loss: 0.0015
Epoch 12/50
 - 7s - loss: 0.0013
Epoch 13/50
 - 7s - loss: 0.0011
Epoch 14/50
 - 7s - loss: 0.0012
Epoch 15/50
 - 7s - loss: 0.0011
Epoch 16/50
 - 8s - loss: 9.6136e-04
Epoch 17/50
 - 8s - loss: 8.3707e-04
Epoch 18/50
 - 9s - loss: 7.6327e-04
Epoch 19/50
 - 9s - loss: 9.2781e-04
Epoch 20/50
 - 9s - loss: 8.2747e-04
Epoch 21/50
 - 8s - loss: 7.1662e-04
Epoch 22/50
 - 7s - loss: 9.2839e-04
Epoch 23/50
 - 7s -

In [16]:
# make predictions 

trainPredict = model.predict(Xtrain, batch_size = batch_size)
testPredict = model.predict(Xtest, batch_size = batch_size)

trainPredict = scaler.inverse_transform(trainPredict)
testPredict = scaler.inverse_transform(testPredict)

ytrain = scaler.inverse_transform([Ytrain])
ytest = scaler.inverse_transform([Ytest])
    
trainScore = sqrt(mean_squared_error(ytrain[0], trainPredict[:,0]))    
print("train Score", trainScore)

testScore = sqrt(mean_squared_error(ytest[0], testPredict[:,0]))    
print("train Score", testScore)
print(ytest[0])

train Score 1.4013521044226716
train Score 9.597689915584928
[ 66.062   69.2555  84.404   84.6447  77.4221  73.4205  67.4636  66.0145
  71.6535  76.8376  77.3265  73.277   67.6237  68.7675  77.9179  86.2914
  77.8027  73.6061  67.3549  69.0228  75.0839  80.3488  79.4052  73.8126
  68.9572  72.7156  79.7391  85.556   80.4178  74.739   69.8367  67.3781
  71.4197  77.7263  76.7791  72.5681  69.2363  73.7175  84.7818  88.2287
  84.7919  79.6885  71.2731  67.9111  74.5503  82.3366  82.2906  74.8323
  69.8758  74.6278  84.4669  94.4017  87.4053  79.4751  71.1817  69.5934
  78.1988  83.5287  82.1321  75.9181  71.4802  75.4754  85.0772  92.7412
  87.667   81.5205  74.0457  73.5003  78.5503  86.2425  89.8063  78.7442
  73.9353  79.8903  91.0933  99.2688  93.1691  87.2075  77.4715  76.9337
  82.1958  86.2321  86.824   79.4425  74.911   81.3     90.753   98.7482
  90.0486  83.2671  76.3793  74.0777  80.0138  88.7168  88.0487  82.1801
  77.7266  82.2802  92.3623  95.0701  88.0838  86.8125  76.9828

In [17]:
trainPredictPlot = np.empty_like(scaled)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

testPredictPlot = np.empty_like(scaled)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(scaled)-1, :] = testPredict




In [18]:
#plot the two values
testPredictPlot = pd.DataFrame(testPredict,index = data[len(trainPredict)+(look_back*2)+1:len(scaled)-1:].index,columns=['Prediction'])
pd.concat([data[len(trainPredict)+(look_back*2)+1:len(scaled)-1],testPredictPlot],axis=1).iplot(title="Prediction",xTitle='TIme Stamp',yTitle='Demand')

In [19]:
# plotting whole dataset with predictions
pd.concat([data,testPredictPlot],axis=1).iplot()


In [20]:
test_samp = Xtest[0:1]
Fulltest = np.array([])

x = model.predict(test_samp)
x[0]

array([0.5205952], dtype=float32)

In [21]:
#  printing r2 score of our model 
rms = sqrt(mean_squared_error(testPredict, ytest[0]))
print(rms)

9.597689915584928


In [22]:
#  printing mean square error of our model 
r2 = r2_score(testPredict,ytest[0])
r2

-0.24755032343322392

In [23]:
# while len(Fulltest) < len(testPredict):
#     x = model.predict(test_samp)
#     np.concatenate(Fulltest.reshape(1, -1), np.array(x[0][0]).reshape(1, -1));
#     test_samp[0] = test_samp[0][1:]
#     test_samp[0] = test_samp[0].concatenate(np.array(x[0][0]))



In [24]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, None, 75)          23100     
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 75)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               70400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 93,601
Trainable params: 93,601
Non-trainable params: 0
_________________________________________________________________
